In [2]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

2023-06-22 13:11:12.067586: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
df = pd.read_csv("../datasets/loan_dataset/loan_train.csv")
df.head()

,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area,Status
0,Male,No,0,Graduate,No,584900,0.0,15000000,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,458300,150800.0,12800000,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,300000,0.0,6600000,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,258300,235800.0,12000000,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,600000,0.0,14100000,360.0,1.0,Urban,Y


Review the data

In [7]:
df.shape

(614, 12)

In [8]:
df.describe()

,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History
count,6.140000e+02,6.140000e+02,6.140000e+02,600.00000,564.000000
mean,5.403459e+05,1.621246e+05,1.414104e+07,342.00000,0.842199
std,6.109042e+05,2.926248e+05,8.815682e+06,65.12041,0.364878
min,1.500000e+04,0.000000e+00,0.000000e+00,12.00000,0.000000
25%,2.877500e+05,0.000000e+00,9.800000e+06,360.00000,1.000000
50%,3.812500e+05,1.188500e+05,1.250000e+07,360.00000,1.000000
75%,5.795000e+05,2.297250e+05,1.647500e+07,360.00000,1.000000
max,8.100000e+06,4.166700e+06,7.000000e+07,480.00000,1.000000


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Gender              601 non-null    object 
 1   Married             611 non-null    object 
 2   Dependents          599 non-null    object 
 3   Education           614 non-null    object 
 4   Self_Employed       582 non-null    object 
 5   Applicant_Income    614 non-null    int64  
 6   Coapplicant_Income  614 non-null    float64
 7   Loan_Amount         614 non-null    int64  
 8   Term                600 non-null    float64
 9   Credit_History      564 non-null    float64
 10  Area                614 non-null    object 
 11  Status              614 non-null    object 
dtypes: float64(3), int64(2), object(7)
memory usage: 57.7+ KB


In [10]:
df.isna().sum()

Gender                13
Married                3
Dependents            15
Education              0
Self_Employed         32
Applicant_Income       0
Coapplicant_Income     0
Loan_Amount            0
Term                  14
Credit_History        50
Area                   0
Status                 0
dtype: int64

We can either drop or fill the missing data. Since this set is so small will opt to fill. 

In [11]:
for i in df.columns:
    df[i].fillna(df[i].mode()[0], inplace = True)

In [12]:
df.isna().sum()

Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
Applicant_Income      0
Coapplicant_Income    0
Loan_Amount           0
Term                  0
Credit_History        0
Area                  0
Status                0
dtype: int64

In [14]:
# just discovered this get_dummies function, for casting categorical data into numerical. Faster than mapping over a dict. 
df = pd.get_dummies(df, columns=['Dependents','Gender','Married','Education','Self_Employed','Area','Status'])

In [16]:
X =  np.array(df.iloc[:,:-2])
y =  np.array(df.iloc[:,-1])

In [17]:
df_test = pd.read_csv("../datasets/loan_dataset/loan_test.csv")
df_test.head()

,Gender,Married,Dependents,Education,Self_Employed,Applicant_Income,Coapplicant_Income,Loan_Amount,Term,Credit_History,Area
0,Male,Yes,0,Graduate,No,572000,0,11000000,360.0,1.0,Urban
1,Male,Yes,1,Graduate,No,307600,150000,12600000,360.0,1.0,Urban
2,Male,Yes,2,Graduate,No,500000,180000,20800000,360.0,1.0,Urban
3,Male,Yes,2,Graduate,No,234000,254600,10000000,360.0,NaN,Urban
4,Male,No,0,Not Graduate,No,327600,0,7800000,360.0,1.0,Urban


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)

In [19]:
minmax = MinMaxScaler()
x_train = minmax.fit_transform(X_train)
x_test = minmax.transform(X_test)

In [20]:
logR = LogisticRegression(solver = 'saga', max_iter = 500, random_state = 1)
logR.fit(x_train, y_train)

LogisticRegression(max_iter=500, random_state=1, solver='saga')

In [21]:
preds = logR.predict(x_test)

In [22]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.94      0.41      0.57        39
           1       0.78      0.99      0.87        84

    accuracy                           0.80       123
   macro avg       0.86      0.70      0.72       123
weighted avg       0.83      0.80      0.78       123



In [23]:
print(accuracy_score(preds,y_test)*100)

80.48780487804879


In [24]:
preds[0]

1

In [30]:
print(df_test.iloc[0])

Gender                    Male
Married                    Yes
Dependents                   0
Education             Graduate
Self_Employed               No
Applicant_Income        572000
Coapplicant_Income           0
Loan_Amount           11000000
Term                     360.0
Credit_History             1.0
Area                     Urban
Name: 0, dtype: object


In [35]:
df_test.isna().sum()

Applicant_Income           0
Coapplicant_Income         0
Loan_Amount                0
Term                       6
Credit_History            29
Dependents_0               0
Dependents_1               0
Dependents_2               0
Dependents_3+              0
Gender_Female              0
Gender_Male                0
Married_No                 0
Married_Yes                0
Education_Graduate         0
Education_Not Graduate     0
Self_Employed_No           0
Self_Employed_Yes          0
Area_Rural                 0
Area_Semiurban             0
Area_Urban                 0
dtype: int64

In [36]:
for i in df_test.columns:
    df_test[i].fillna(df_test[i].mode()[0], inplace = True)

In [32]:
df_test = pd.get_dummies(df_test, columns=['Dependents','Gender','Married','Education','Self_Employed','Area'])

In [38]:
test_scaled = minmax.fit_transform(df_test)

In [39]:
test_preds = logR.predict(test_scaled)

In [40]:
test_preds[0]

1